#  (Nguyen Tuong Ha - nguyenpanda)
- Dataset link: [Kaggle]()

# Table of Contents
<!-- TOC -->

<!-- TOC -->

# 0. Setup

## 0.0. Install packages

In [ ]:
!pip install nguyenpanda kaggle torchmetrics torchinfo mlxtend

In [ ]:
import torch
import torchvision

from torch import nn
from torch import optim
from torchmetrics import ConfusionMatrix
from torch.utils import data as torch_data
from torch.utils.data import DataLoader, Dataset, random_split
from torch.utils.data.dataloader import default_collate

from torchvision import datasets
from torchvision import transforms

from torchinfo import summary

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import mlxtend as mle
import nguyenpanda

import os
import sys
import pathlib
import importlib
import random
import typing_extensions as te

from nguyenpanda.swan import color, green, yellow
import nguyenpanda.crow as crow
from PIL import Image
from IPython import display
from tqdm.auto import tqdm
from mlxtend.plotting import plot_confusion_matrix

from pathlib import Path

## 0.1. Checking running environment 

In [ ]:
DEVICE = ('cuda' if torch.cuda.is_available() else
          'mps' if torch.backends.mps.is_available() else
          'cpu')
print('DEVICE:', green(DEVICE))

CPUs = os.cpu_count()
print('CPUs:', green(CPUs))

# print(plt.style.available)
plt.style.use('dark_background')

## 0.2. Download datasets on Kaggle if using Google Colab

In [ ]:
LOCALLY_DOWNLOAD: bool = False

# DATASET_DIR_NAME
#   - Local: replace it with actual directory name
#   - Google Colab: name what you want "dataset_alias" or "dataset"
DATASET_DIR_NAME = '<Name>'

# KAGGLE_API_COMMAND
#   - Local: This can be ignored
#   - Google Colab: replace it by actual API command
#       → Example: "kaggle datasets download -d muhammadehsan000/diabetes-healthcare-dataset"
KAGGLE_API_COMMAND = ''
DATASET_SOURCE_DIR: Path = Path('/Users/hatuongnguyen/datasets') / DATASET_DIR_NAME

In [ ]:
IN_COLAB: bool = crow.nbu.is_colab()
if IN_COLAB:
    from google.colab import files, output, drive

    DATASET_SOURCE_DIR: Path = Path.cwd() / DATASET_DIR_NAME

    print(yellow('Please upload your Kaggle API json file'))
    output.eval_js('new Audio("https://static.wikia.nocookie.net/soundeffects/images/c/c8/IOS_Message_Tone_Sound.ogg").play()')
    files.upload()
    os.system('mkdir ~/.kaggle')
    os.system('cp kaggle.json ~/.kaggle/')
    os.system('chmod 600 ~/.kaggle/kaggle.json')

if IN_COLAB or LOCALLY_DOWNLOAD:
    print(green('Downloading Kaggle dataset...'))
    if len(KAGGLE_API_COMMAND) < 29:
        raise ValueError('KAGGLE_API_COMMAND must be replaced by actual API command')
    os.system(KAGGLE_API_COMMAND)
    print(green('Unzip Kaggle Dataset...'))
    os.system(f'unzip {KAGGLE_API_COMMAND.split("/")[-1]}.zip -d "{DATASET_DIR_NAME}"')
    
if IN_COLAB:
    print(yellow('Connect to Google Drive to save model Parameter and Figure, you can delete code below if you don\'t want'))
    output.eval_js('new Audio("https://static.wikia.nocookie.net/soundeffects/images/c/c8/IOS_Message_Tone_Sound.ogg").play()')
    drive.mount('/content/drive')

In [ ]:
print(f'DATASET_SOURCE_DIR:', green(DATASET_SOURCE_DIR))
DATASET_DIR: Path = crow.nb_utils.create_alias(DATASET_SOURCE_DIR, DATASET_DIR_NAME)
print(f'DATASET_DIR:', green(DATASET_DIR))